In [0]:
#from google.colab import drive
#drive.mount('/content/drive')

data_dir = '/content/drive/My Drive/cs461/data'

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

In [0]:
class Args:
  def __init__(self):
    self.use_cuda = True
    self.log_interval = 1
    self.train_batch_size = 64
    self.test_batch_size = 64
    self.lr = 0.01
    self.momentum = 0.9
    self.num_epochs = 3

args = Args()

In [7]:
device = torch.device('cpu')
if args.use_cuda and torch.cuda.is_available():
  device = torch.device('cuda:0')

print('Using {}.'.format(device))

Using cuda:0.


In [0]:
def prepare_data(args):
  kwargs = {}
  if args.use_cuda and torch.cuda.is_available():
    kwargs = {'num_workers': 1, 'pin_memory': True}

  mean = (0.485, 0.456, 0.406)
  std = (0.229, 0.224, 0.225)

  train_transform = transforms.Compose([
      transforms.Lambda(lambda x: x.convert('RGB')),
      transforms.RandomResizedCrop(224),
      transforms.RandomHorizontalFlip(),
      transforms.ToTensor(),
      transforms.Normalize(mean, std),
  ])
  test_transform = transforms.Compose([
      transforms.Lambda(lambda x: x.convert('RGB')),
      transforms.Resize(256),
      transforms.CenterCrop(224),
      transforms.ToTensor(),
      transforms.Normalize(mean, std),
  ])

  # Load original data set with training transforms.
  dataset = torchvision.datasets.Caltech101(
      root=data_dir, download=True, transform=train_transform)
  
  n = len(dataset)
  train_indices = [i for i in range(n) if i % 5 != 0 and i % 5 != 1]
  val_indices = [i for i in range(n) if i % 5 == 1]
  test_indices = [i for i in range(n) if i % 5 == 0]

  # Train.
  train_set = torch.utils.data.Subset(dataset, train_indices)
  train_loader = torch.utils.data.DataLoader(
      train_set, batch_size=args.train_batch_size, shuffle=True, **kwargs)
  
  # Load original data set with testing transforms.
  dataset = torchvision.datasets.Caltech101(
      root=data_dir, download=False, transform=test_transform)

  # Val.
  val_set = torch.utils.data.Subset(dataset, val_indices)
  val_loader = torch.utils.data.DataLoader(
      val_set, batch_size=args.test_batch_size, shuffle=False, **kwargs)
  
  # Test.
  test_set = torch.utils.data.Subset(dataset, test_indices)
  test_loader = torch.utils.data.DataLoader(
      test_set, batch_size=args.test_batch_size, shuffle=False, **kwargs)
  
  return train_set, val_set, test_set, train_loader, val_loader, test_loader

In [9]:
train_set, val_set, test_set, train_loader, val_loader, test_loader = prepare_data(args)

0it [00:00, ?it/s]

100%|█████████▉| 131530752/131740031 [00:16<00:00, 9517562.41it/s]

Extracting /content/drive/My Drive/cs461/data/caltech101/101_ObjectCategories.tar.gz to /content/drive/My Drive/cs461/data/caltech101



0it [00:00, ?it/s]


 95%|█████████▌| 13377536/14028800 [00:03<00:00, 5414132.73it/s]

Extracting /content/drive/My Drive/cs461/data/caltech101/101_Annotations.tar to /content/drive/My Drive/cs461/data/caltech101


Define the training function.

In [0]:
def train(args, model, criterion, train_loader, optimizer, device):
  model.train()
  total_loss = 0.
  for i, data in enumerate(train_loader):
    imgs, lbls = data[0].to(device), data[1].to(device)

    optimizer.zero_grad()
    outputs = model(imgs)
    loss = criterion(outputs, lbls)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()
    if (i + 1) % args.log_interval == 0:
      mean_loss = total_loss / args.log_interval
      print('  batch {:4d}: loss={:.3f}'.format(i + 1, mean_loss))
      total_loss = 0.

Define the testing function.

In [11]:
def test(args, model, test_loader, device):
  model.eval()
  total, correct = 0, 0
  with torch.no_grad():
    for data in test_loader:
      imgs, lbls = data[0].to(device), data[1].to(device)
      outputs = model(imgs)
      _, preds = torch.max(outputs.data, 1)
      total += lbls.shape[0]
      correct += (preds == lbls).sum().item()

  acc = correct / total
  print('  acc={:.3f}'.format(acc))


14032896it [00:20, 5414132.73it/s]                              

Run training and validation.

In [16]:
model = torchvision.models.mobilenet_v2(pretrained=True)
model.classifier[-1] = nn.Linear(model.last_channel, 101)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(
    model.parameters(),
    lr=args.lr,
    momentum=args.momentum
)

args.lr = 0.005
args.num_epochs = 50

for e in range(args.num_epochs):
  print('Training epoch {}'.format(e))
  train(args, model, criterion, train_loader, optimizer, device)
  print('Testing on validation set')
  test(args, model, val_loader, device)

test(args, model, test_loader, device)

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/checkpoints/mobilenet_v2-b0353104.pth


  0%|          | 0.00/13.6M [00:00<?, ?B/s]

 53%|█████▎    | 7.19M/13.6M [00:00<00:00, 75.4MB/s]

100%|██████████| 13.6M/13.6M [00:00<00:00, 85.2MB/s]

Training epoch 0
  batch    1: loss=4.709
  batch    2: loss=4.617
  batch    3: loss=4.690
  batch    4: loss=4.437
  batch    5: loss=4.321
  batch    6: loss=4.100
  batch    7: loss=4.224
  batch    8: loss=3.464
  batch    9: loss=3.558
  batch   10: loss=3.634
  batch   11: loss=3.670
  batch   12: loss=3.510
  batch   13: loss=3.569
  batch   14: loss=3.395
  batch   15: loss=3.562
  batch   16: loss=2.827
  batch   17: loss=3.026
  batch   18: loss=3.367
  batch   19: loss=2.984
  batch   20: loss=2.737
  batch   21: loss=3.318
  batch   22: loss=3.693
  batch   23: loss=3.256
  batch   24: loss=2.967
  batch   25: loss=2.972
  batch   26: loss=3.447
  batch   27: loss=2.661
  batch   28: loss=3.007
  batch   29: loss=2.804
  batch   30: loss=2.590
  batch   31: loss=3.261
  batch   32: loss=2.531
  batch   33: loss=2.364
  batch   34: loss=2.690
  batch   35: loss=2.452
  batch   36: loss=2.607
  batch   37: loss=2.461
  batch   38: loss=2.977
  batch   39: loss=2.651
  batch 

In [14]:
raw_model = torchvision.models.mobilenet_v2(pretrained=False)
raw_model.classifier[-1] = nn.Linear(raw_model.last_channel, 101)
raw_model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(
    raw_model.parameters(),
    lr=args.lr,
    momentum=args.momentum
)

args.lr = 0.005
args.num_epochs = 100

for e in range(args.num_epochs):
  print('Training epoch {}'.format(e))
  train(args, raw_model, criterion, train_loader, optimizer, device)
  print('Testing on validation set')
  test(args, raw_model, val_loader, device)

Training epoch 0
  batch    1: loss=4.609
  batch    2: loss=4.563
  batch    3: loss=4.571
  batch    4: loss=4.326
  batch    5: loss=4.332
  batch    6: loss=4.311
  batch    7: loss=4.400
  batch    8: loss=4.505
  batch    9: loss=4.222
  batch   10: loss=4.000
  batch   11: loss=4.667
  batch   12: loss=4.481
  batch   13: loss=4.397
  batch   14: loss=4.055
  batch   15: loss=4.144
  batch   16: loss=4.146
  batch   17: loss=4.109
  batch   18: loss=4.128
  batch   19: loss=3.968
  batch   20: loss=4.275
  batch   21: loss=4.285
  batch   22: loss=4.090
  batch   23: loss=4.141
  batch   24: loss=4.208
  batch   25: loss=4.191
  batch   26: loss=4.244
  batch   27: loss=4.184
  batch   28: loss=4.513
  batch   29: loss=4.271
  batch   30: loss=4.273
  batch   31: loss=4.143
  batch   32: loss=4.366
  batch   33: loss=4.086
  batch   34: loss=4.380
  batch   35: loss=3.912
  batch   36: loss=4.282
  batch   37: loss=4.155
  batch   38: loss=3.963
  batch   39: loss=3.815
  batch 

In [15]:
test(args, raw_model, test_loader, device)

  acc=0.732
